In [8]:
import sys
import os
import pytz
from datetime import datetime

import psycopg2
import pandas as pd
import numpy as np

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))
from constants import itemtype, currency, league

league_id = league.BREACH
MIN_PRICE = 1  # Anything below this (in chaos) is considered worthless
FRESH_SECONDS = 24*60*60  # Assume undercut items worth MIN_PRICE will be bought in this time

In [9]:
rates = currency.get_exchange_rates(league.get_name(league_id))
print(rates)

def get_chaos_price(price, currency):
    return price * rates[currency]

def is_item_valuable(price, currency, added_time, sold_time):
    """
    Checks if an item is valuable or not.
    May also return None if we can't tell.
    """
    fresh = (datetime.now(tz=pytz.UTC) - added_time).total_seconds() < FRESH_SECONDS
    sold = sold_time > datetime(2000, 1, 1, tzinfo=pytz.UTC)
    price = get_chaos_price(price, currency)
    
    # If item was offered for a low price for more than a day, assume it's worthless
    if price < MIN_PRICE and not fresh:
        return False
        
    # If item is offered for a low price but the offer is younger than a day, ignore it
    if price < MIN_PRICE and fresh:
        return None
        
    # If item is sold for less than min price, ignore it 
    # because we don't know if the buyer would have also bought it at a higher price
    if price < MIN_PRICE and sold:
        return None
        
    # If item was sold at min price or higher, it's valuable
    if price >= MIN_PRICE and sold:
        return True
    
    # If it's not sold and price is higher than min price, it might just be overpriced
    return None
    

dbconn = psycopg2.connect(dbname='poeria')
db = dbconn.cursor()

{1: 0.07, 2: 0.32, 3: 0.28, 4: 1.0, 5: 0.94, 6: 72.29, 7: 0.07, 8: 0.07, 9: 0.16, 10: 0.29, 11: 0.65, 12: 0.23, 13: 1.03, 14: 0.95, 15: 13.5, 16: 1.43}


# Rings

In [14]:
db.execute("""
SELECT Price, Currency, AddedTime, SoldTime, -- 3
       Corrupted, Sockets, DoubledInBreach,  -- 6
       Strength, Dexterity, Intelligence,    -- 9
       Life, Mana, Evasion, EnergyShield,    -- 13
       Accuracy, CritChance, AttackSpeed, CastSpeed,   -- 17
       FireResist, ColdResist, LightningResist, ChaosResist,  -- 21
       AddedPhysAttackDamage, AddedFireAttackDamage, AddedColdAttackDamage,  --24
       AddedLightningAttackDamage, AddedChaosAttackDamage,  -- 26                                                  -- 31
       IncreasedEleDamage, IncreasedWeaponEleDamage, IncreasedFireDamage, IncreasedColdDamage, IncreasedLightningDamage, 
       ItemRarity, LifeLeech, ManaLeech, LifeGainOnHit, LifeGainOnKill, LifeRegen, ManaGainOnKill, ManaRegen,   -- 39
       AvoidFreeze, GrantedSkillId, ManaGainOnHit, DamageToMana, LightRadius  --44
  FROM StashContents s, RingItems r
 WHERE s.league = %s
   AND s.ItemId = r.ItemId""",
           (league_id,))

features = []
targets = []

num_valuable = 0
num_worthless = 0

for row in db:
    valuable = is_item_valuable(row[0], row[1], row[2], row[3])
    
    # Ignore this item if we can't tell if it's valuable or not
    if valuable is None:
        continue
        
    targets.append(1 if valuable else 0)
    features.append({
        'Corrupted': row[4],
        'HasSocket': len(row[5]),
        'HasWhiteSocket': 1 if row[5] == 'W' else 0, 
        'DoubledInBreach': row[6],  
        'Strength': row[7],  
        'Dexterity': row[8],  
        'Intelligence': row[9],  
        'Life': row[10],  
        'Mana': row[11],  
        'Evasion': row[12],  
        'EnergyShield': row[13],  
        'Accuracy': row[14],  
        'CritChance': row[15],  
        'AttackSpeed': row[16],  
        'CastSpeed': row[17],  
        'TotalEleResist': row[18] + row[19] + row[20],  
        'ChaosResist': row[21],  
        'AddedPhysAttackDamage': row[22],  
        'AddedFireAttackDamage': row[23],  
        'AddedColdAttackDamage': row[24],  
        'AddedLightningAttackDamage': row[25],  
        'AddedChaosAttackDamage': row[26],  
        'IncreasedEleDamage': row[27],  
        'IncreasedWeaponEleDamage': row[28],  
        'IncreasedFireDamage': row[29],  
        'IncreasedColdDamage': row[30],  
        'IncreasedLightningDamage': row[31],  
        'ItemRarity': row[32],  
        'LifeLeech': row[33],  
        'ManaLeech': row[34],  
        'LifeGainOnHit': row[35],  
        'LifeGainOnKill': row[36],  
        'LifeRegen': row[37],  
        'ManaGainOnKill': row[38],  
        'ManaRegen': row[39],  
        'ManaGainOnHit': row[40],  
        'DamageToMana': row[41],  
        'LightRadius': row[42],   
    })
    
    num_valuable += targets[-1]
    num_worthless += 1 - targets[-1]

print("Found a total of {} interesting items".format(len(features)))
print("    {:10} valuable ({:.1f}%)".format(num_valuable, 100*(num_valuable/len(features))))
print("    {:10} worthless ({:.1f}%)".format(num_worthless, 100*(num_worthless/len(features))))
print()

features = pd.DataFrame(features, dtype=int)
targets = pd.DataFrame(data=targets, columns=['valuable'])

features_max = features.max()#.map(lambda x: 1 if x == 0 else x)
print(features_max)
print()
features_max.to_csv('ring_max.csv')

#features = features.divide(features_max)
features.to_csv('ring_features.csv.bz2', compression='bz2')

targets.to_csv('ring_targets.csv')

Found a total of 214304 interesting items
         82394 valuable (38.4%)
        131910 worthless (61.6%)

Accuracy                      439
AddedChaosAttackDamage         22
AddedColdAttackDamage          62
AddedFireAttackDamage          70
AddedLightningAttackDamage     83
AddedPhysAttackDamage          46
AttackSpeed                     7
CastSpeed                       7
ChaosResist                    48
Corrupted                       1
CritChance                     56
DamageToMana                    6
Dexterity                      56
DoubledInBreach                 1
EnergyShield                   72
Evasion                       170
HasSocket                       1
HasWhiteSocket                  0
IncreasedColdDamage            33
IncreasedEleDamage             67
IncreasedFireDamage            34
IncreasedLightningDamage       34
IncreasedWeaponEleDamage       42
Intelligence                   57
ItemRarity                     63
Life                          136
LifeGain